In [2]:
import spacy
from spacy import displacy

import en_core_web_sm

import pandas as pd

from pathlib import Path

from nltk.tokenize import sent_tokenize

import re

import numpy as np



from nltk import stem


In [3]:
def newDescriptions(descriptions):
    newDescr=[]
    for d in descriptions: 
        try:
            new_d = stripPunct(d)
            new_d = makeLower(new_d)
            if isVerb(d):
                x = "This repository " + str(new_d)
            else:
                x = "This repository contains " + str(new_d)
        except:
            x = ""
        newDescr.append(x)
    return newDescr

In [4]:
def isVerb(description):
    doc = nlp(description)
    
    if doc[0].pos_ == 'VERB':
        return True
    else:
        return False

In [5]:
def getTree(text):

    df_tree = pd.DataFrame(columns=['Text','Dep','Head Text','Head Pos','Children', 'Stop','Lemma/Stem'])
    stemmer=stem.PorterStemmer()
    if type(text) is str:
        doc = nlp(text)
        for token in doc:

            df_tree = df_tree.append({'Text':token.text, 'Dep':token.dep_, 'Head Text':token.head.text,
                           'Head Pos':token.head.pos_, 'Children':[child for child in token.children], 'Stop':token.is_stop, 'Lemma/Stem':stemmer.stem(token.lemma_)}, ignore_index=True)

    return df_tree


def getAllTrees(texts):
    trees = []
    for text in texts:
        trees.append(getTree(text))
        
    return trees



In [6]:
# strip punctuation

In [7]:
# define punctuation
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def stripPunct(my_str):


    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
        else:
            no_punct = no_punct + " "
    return no_punct

def stripAllPunct(strings):
    to_return=[]
    for s in strings:
        try:
            to_return.append(stripPunct(s))
        except:
            to_return.append(stripPunct(s))
    return to_return

def makeLower(my_str):
    return my_str.lower()

def makeAllLower(strings):
    to_return=[]
    for s in strings:
        try:
            to_return.append(makeLower(s))
        except:
            to_return.append("")
    return to_return

In [8]:
def incrementOccurance(index, occurances):
    occurances[index]+=1
    
def singleTreeOccurance(lst, list_of_universal_dependencies, occurances):
    for d in lst:
        try:
            index=list_of_universal_dependencies.index(str(d))
        except:
            list_of_universal_dependencies.append(str(d))
            index = list_of_universal_dependencies.index(str(d))
            
       
        incrementOccurance(index, occurances)
        
        
def allTreeOccurances(lists, list_of_universal_dependencies, occurances):
    for l in lists:
        singleTreeOccurance(l, list_of_universal_dependencies, occurances)
        
        


In [9]:
# # for validation

# occurances_v=np.zeros((36,1))
# list_of_universal_dependencies_v=[]
# example_dependencies_v=['None']*36
# tree_occurance_v = allTreeOccurances(trees_v, list_of_universal_dependencies_v, occurances_v)

# # for utility

# occurances_u=np.zeros((37,1))
# list_of_universal_dependencies_u=[]
# example_dependencies_u=['None']*37
# tree_occurance_u = allTreeOccurances(trees_u, list_of_universal_dependencies_u, occurances_u)

# # for organization

# occurances_o=np.zeros((36,1))
# list_of_universal_dependencies_o=[]
# example_dependencies_o=['None']*36
# tree_occurance_o = allTreeOccurances(trees_o, list_of_universal_dependencies_o, occurances_o)

In [10]:
# it_v = 0
# for i in occurances_v:
    
#     #print(i, list_of_universal_dependencies[it], spacy.explain(list_of_universal_dependencies[it]))
#     it_v+=1
    
# o_v = []    
# for i in occurances_v:
#     o_v.append(int(i))
    
# print(len(list_of_universal_dependencies_v))




# #######################
# it_u = 0
# for i in occurances_u:
    
#     #print(i, list_of_universal_dependencies[it], spacy.explain(list_of_universal_dependencies[it]))
#     it_u+=1
    
# o_u = []    
# for i in occurances_u:
#     o_u.append(int(i))
    
# print(len(list_of_universal_dependencies_u))





# #######################
# it_o = 0
# for i in occurances_o:
    
#     #print(i, list_of_universal_dependencies[it], spacy.explain(list_of_universal_dependencies[it]))
#     it_o+=1
    
# o_o = []    
# for i in occurances_o:
#     o_o.append(int(i))
    
# print(len(list_of_universal_dependencies_o))

In [11]:
# take first 80 descriptions
# remove stop words
# stem words (or lematize ... figure out which is better)
# import the purpose from the correct sheet
# stem or lematize
# match individual words in description to purpose and save dep, head text, pos for properly matched words

# next look at intro paragraph of readme to do the same.

# alternatively, for the properly id'd words ---> get first sentence from wikipedia for background (?)

In [15]:
def lemmaAndStem(tree):
    stemmer=stem.PorterStemmer()
    string = ""
    count=0
    lemma = tree['Lemma/Stem']
    for l in lemma:
        if count==0:
            string = string+stemmer.stem(l)
            count+=1
        else:
            string = string + " " +  stemmer.stem(l)
    return string
    

def allLemmaStem(trees):
    lems=[]
    for t in trees:
        try:
            l=(lemmaAndStem(t))
        except:
            l=""
        lems.append(l)
    return lems


In [16]:
import difflib

In [17]:
# find all matching sequences using difflib library

def matchTwoStrings(string1, string2):
    to_return=[]
    matches = difflib.SequenceMatcher(lambda x: x == " ",string1,string2).get_matching_blocks()
    for match in matches:
        x=(string1[match.a:match.a+match.size])
        if len(x)<2:
            continue
        else:
            to_return.append(x)
    return to_return


def findAllMatches(list1,list2):
    to_return=[]
    for i in range(len(list1)):
        matches=matchTwoStrings(list1[i],list2[i])
        to_return.append(matches)
        
    return to_return

In [18]:
# from matching sequences, get dependency structure

def matchTwoStrings(string1, string2):
    to_return=[]
    matches = difflib.SequenceMatcher(lambda x: x == " ",string1,string2).get_matching_blocks()
    for match in matches:
        x=(string1[match.a:match.a+match.size])
        if len(x)<3:
            continue
        else:
            to_return.append(x)
    return to_return


def findAllMatches(list1,list2):
    to_return=[]
    for i in range(len(list1)):
        matches=matchTwoStrings(list1[i],list2[i])
        to_return.append(matches)
        
    return to_return

In [20]:



def findIndexInSeries(series,value):
    index=0
    for i in range(len(series)):
        if str(value)!=series[i]:
            index+=1
        else:
            return index


def findCorrelatingDep(dataframe,index, token):
    #print(index)
    try:
        df_temp=dataframe.iloc[[index]]
        dep = df_temp["Dep"]
        for i in dep:
            return(i)
    except:
        return("*"+token+"*")

def setSeriesPurposeTrue(series, idx):
#     tree.at[idx,'Purpose']=1
#

#      To finish. 
#      goal: create new series of 0's and 1's representing which index of initial dataframe is purpose and which isnt


#
    pass
    
        
    
def matchesPerTree(matches, tree):
    dependencies=[]
    for match in matches:
        doc=nlp(match)
        for token in doc:
            if str(token) == " ":
                continue
            else:
                idx = findIndexInSeries(tree["Lemma/Stem"], str(token))
                dependency = findCorrelatingDep(tree,idx, str(token))
                #setPurposeTrue(tree, idx)
                #print(tree)
                dependencies.append(dependency)
    return dependencies

def allTreeMatches(listOfMatches,listOfTrees):
    listOfDependencies=[]
    for i in range(len(listOfMatches)):
        deps = matchesPerTree(listOfMatches[i],listOfTrees[i])
        listOfDependencies.append(deps)
    return listOfDependencies

In [19]:
# load file to update
file_v = "validation dataset/validation_info_init.csv"
file_o = "organization dataset/organization_info.csv"
file_u = "utility dataset/utility_info.csv"
sheet_v = pd.read_csv(file_v, encoding = "ISO-8859-1")
sheet_o = pd.read_csv(file_o)
sheet_u = pd.read_csv(file_u)
nlp = en_core_web_sm.load()

descriptions_v = sheet_v['Description']
descriptions_o = sheet_o['Description']
descriptions_u = sheet_u['Description']


#clean purpose (lower case and remove punctuation)
purpose_v = sheet_v['Purpose']
purpose_v = makeAllLower(purpose_v)
purpose_v = stripAllPunct(purpose_v)


#clean description (lower case and remove punctuation) and make a sentence
newD_v = newDescriptions(descriptions_v)
newD_o = newDescriptions(descriptions_o)
newD_u = newDescriptions(descriptions_u)

# make into dataframe
df_newDescr_v = pd.DataFrame(newD_v)
df_newDescr_o = pd.DataFrame(newD_o)
df_newDescr_u = pd.DataFrame(newD_u)

df_purpose_v=pd.DataFrame(purpose_v)


# get the tree structure for each sentence/purpose
trees_v = getAllTrees(newD_v)
trees_o = getAllTrees(newD_o)
trees_u = getAllTrees(newD_u)
trees_purpose = getAllTrees(purpose_v)


# take first 80 from validation of both sentence and purpose
trees_match = trees_v[0:81]
trees_match_purpose = trees_purpose[0:81]

descr_match=newD_v[0:81]

lems_descr = allLemmaStem(trees_match)

lems_purpose = allLemmaStem(trees_match_purpose)



# find all string matches between initial description and manually labeled purposes

matches=findAllMatches(lems_descr,lems_purpose)

In [21]:
# for each description's list of matched strings, find the corresponding dependency type
# trees_match
dependenciesMatched=allTreeMatches(matches, trees_match)

In [22]:
# for i in range(len(dependenciesMatched)):
#     print(i, " : ", dependenciesMatched[i], "\n\t", matches[i],"\n\t",trees_match[i], trees_match_purpose[i])

# # #dataframe_matched = pd.DataFrame(columns=['Number'])

dependenciesMatched

[['compound', 'dobj', 'prep', 'compound', 'pobj', 'compound'],
 ['amod', 'pobj'],
 ['nsubj', 'ROOT', 'conj'],
 [],
 ['xcomp', 'dobj'],
 ['*cont*'],
 ['compound', 'dobj'],
 ['compound', 'nsubj', 'amod', 'pobj'],
 ['amod', 'amod', 'ROOT'],
 [],
 ['relcl', 'compound', '*protoc*'],
 ['prep', 'compound', 'pobj'],
 ['compound', 'compound', 'dobj'],
 ['amod', 'pobj', 'prep', '*quickba*'],
 [],
 [],
 ['nsubj'],
 ['compound', '*pro*'],
 [],
 ['compound'],
 ['compound'],
 ['*re*', 'amod'],
 ['dobj', 'cc', 'compound', 'compound'],
 ['*co*'],
 [],
 ['compound', 'ROOT'],
 ['xcomp', 'pobj'],
 ['dobj', 'compound', 'pobj'],
 ['xcomp', 'dobj'],
 ['acl', 'compound', 'dobj'],
 [],
 [],
 ['compound', 'prep', 'pobj'],
 ['amod', 'compound', 'prep', 'pobj'],
 [],
 ['dobj', 'prep', 'pobj', 'compound', 'dobj'],
 ['attr', 'prep', 'amod', 'compound'],
 ['*l*', 'prep', 'amod', 'pobj'],
 ['ROOT', 'compound', 'dobj', 'prep', 'compound', 'pobj'],
 [],
 ['compound'],
 ['compound', 'compound', 'compound', 'dobj'],
 []

In [1]:
print(lems_purpose[49])
print(lems_descr[49])
print(matches[49])

NameError: name 'lems_purpose' is not defined

In [196]:
occurances_purpose=np.zeros((42,1))
list_of_universal_dependencies_purpose=[]
example_dependencies_purpose=['None']*42
tree_occurance_purpose = allTreeOccurances(dependenciesMatched, list_of_universal_dependencies_purpose, occurances_purpose)

In [200]:
for i in range(len(occurances_purpose)):
    if(spacy.explain(list_of_universal_dependencies_purpose[i])is not None):
        print(i, " : " , occurances_purpose[i], (list_of_universal_dependencies_purpose[i]), " = " ,spacy.explain(list_of_universal_dependencies_purpose[i]))

0  :  [43.] compound  =  compound
1  :  [28.] dobj  =  direct object
2  :  [17.] prep  =  prepositional modifier
3  :  [24.] pobj  =  object of preposition
4  :  [25.] amod  =  adjectival modifier
5  :  [3.] nsubj  =  nominal subject
7  :  [1.] conj  =  conjunct
8  :  [7.] xcomp  =  open clausal complement
10  :  [1.] relcl  =  relative clause modifier
15  :  [2.] cc  =  coordinating conjunction
17  :  [1.] acl  =  clausal modifier of noun (adjectival clause)
18  :  [2.] attr  =  attribute
22  :  [1.] aux  =  auxiliary
23  :  [4.] det  =  determiner
24  :  [1.] advmod  =  adverbial modifier
28  :  [2.] ccomp  =  clausal complement
31  :  [1.] npadvmod  =  noun phrase as adverbial modifier
33  :  [1.] nummod  =  numeric modifier
38  :  [1.] nmod  =  modifier of nominal
40  :  [2.] pcomp  =  complement of preposition
